In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as mpl_anim
from IPython.display import HTML
from qutip import *

# 1. Setup Simulation (Coupled Cavities)
N = 10              
w = 1.0           
J = 0.05            
H_a = w * tensor(destroy(N), qeye(N)).dag() * tensor(destroy(N), qeye(N))
H_b = w * tensor(qeye(N), destroy(N)).dag() * tensor(qeye(N), destroy(N))
H_hop = J * (tensor(destroy(N), qeye(N)).dag() * tensor(qeye(N), destroy(N)) + 
             tensor(destroy(N), qeye(N)) * tensor(qeye(N), destroy(N)).dag())

H = H_a + H_b + H_hop

# Initial State: Photon in A, Vacuum in B
psi0 = tensor(basis(N, 1), basis(N, 0))

# Time Evolution (One full swap: A -> B)
t_swap = np.pi / (2 * J)
frames = 60
tlist = np.linspace(0, t_swap, frames)

print("1. Running Hopping Simulation...")
output = mesolve(H, psi0, tlist, [], [])

# Pre-Calculate Wigner Data for BOTH Cavities
print("2. Calculating Wigner frames for both cavities...")
xvec = np.linspace(-5, 5, 80) 
wigner_A = []
wigner_B = []

for state in output.states:
    # Trace out B to see A
    rho_A = state.ptrace(0)
    wigner_A.append(wigner(rho_A, xvec, xvec))
    
    # Trace out A to see B
    rho_B = state.ptrace(1)
    wigner_B.append(wigner(rho_B, xvec, xvec))

# Build the Animation
print("3. Building Animation...")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Fixed color scale
w_max = np.max(wigner_A[0])
w_min = -w_max

# Initial Plots
mesh1 = ax1.contourf(xvec, xvec, wigner_A[0], 100, cmap="RdBu_r", levels=np.linspace(w_min, w_max, 100))
mesh2 = ax2.contourf(xvec, xvec, wigner_B[0], 100, cmap="RdBu_r", levels=np.linspace(w_min, w_max, 100))

ax1.set_title("Cavity A (Sender)")
ax2.set_title("Cavity B (Receiver)")
ax1.set_aspect('equal')
ax2.set_aspect('equal')

def update(frame):
    ax1.clear()
    ax2.clear()
    
    # Update Cavity A
    ax1.contourf(xvec, xvec, wigner_A[frame], 100, cmap="RdBu_r", levels=np.linspace(w_min, w_max, 100))
    ax1.set_title("Cavity A (Sender)")
    
    # Update Cavity B
    ax2.contourf(xvec, xvec, wigner_B[frame], 100, cmap="RdBu_r", levels=np.linspace(w_min, w_max, 100))
    ax2.set_title("Cavity B (Receiver)")
    
    # Add Time Label
    fig.suptitle(f"Photon Hopping t={tlist[frame]:.2f}", fontsize=14)
    return ax1, ax2

anim = mpl_anim.FuncAnimation(fig, update, frames=frames, interval=100, blit=False)

plt.close()
HTML(anim.to_jshtml())

1. Running Hopping Simulation...
2. Calculating Wigner frames for both cavities...


c:\Users\Vedant\miniconda3\envs\qiskit-ok\lib\site-packages\qutip\solver\solver_base.py:598: FutureWarning: e_ops will be keyword only from qutip 5.3 for all solver
  warnings.warn(


3. Building Animation...
